In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import akshare as ak
import statistics as stats

warnings.filterwarnings('ignore')

In [25]:
class MonteCarlo:
                    def __init(self,port_weights, inital_prices, mu_vector, sigma_vector, corr_matrix, T,n_simulations, alpha):
                        """
                        Args
                            :param port_weights: 组合权重数组
                            :param inital_prices: 初始价格数组
                            :param mu_vector: 收益率均值数组
                            :param sigma_vector: 波动率数组
                            :param corr_matrix: 相关系数矩阵
                            :param T: 持有期（日）
                        """
                        self.weights = np.array(port_weights)
                        self.S0 = np.array(inital_prices)
                        self.mu = np.array(mu_vector)
                        self.sigma = np.array(sigma_vector)
                        self.corr_matrix = np.array(corr_matrix)
                        self.T = T
                        self.n_assets = len(port_weights)
                        self.n_simulations = n_simulations
                        self.alpha = alpha

                        #检查维度一致性
                        assert len(self.weights) == self.n_assets
                        assert len(self.S0) == self.n_assets
                        assert len(self.mu) == self.n_assets
                        assert  len(self.sigma) == self.n_assets
                        assert self.corr_matrix.shape == (self.n_assets, self.n_assets)

                        #计算协方差矩阵
                        self.cov_matrix = np.diag(self.sigma) @ self.corr_matrix @ np.diag(self.sigma)

                    def simulate_multivariate_gbm(self, random_speed=42):
                        np.random.seed(random_speed)
                        n_steps = self.T

                        #使用Cholesky分解处理相关性
                        L = np.linalg.cholesky(self.cov_matrix)

                        #生成独立随机数
                        Z = np.random.normal(0,1,(n_steps, self.n_simulations, self.n_assets))

                        #应用相关性变换
                        Z_correlated = np.zeros_like(Z)
                        for t in range(n_steps):
                            Z_correlated[t] = Z[t] @ L.T

                        #模拟价格路径
                        price_paths = np.zeros((n_steps+1, self.n_simulations, self.n_assets))
                        price_paths[0] = self.S0

                        dt = 1/252
                        drift = (self.mu - 0.5 * self.sigma**2) * dt

                        for t in range(1, n_steps+1):
                            diffusion = self.sigma * np.sqrt(dt)
                            growth_factors = np.exp(drift+diffusion*Z_correlated[t-1])
                            price_paths[t] = price_paths[t-1] * growth_factors

                        self.price_paths = price_paths

                        return price_paths

                    def calculate_port_var(self, condidence_level=1-alpha):
                        if not hasattr(self, 'price_paths'):
                            self.simulate_multivariate_gbm()
                        port_values = np.sum(self.price_paths * self.weights, axis=2)

                        initial_value = np.sum(self.S0 * self.weights)
                        final_values = port_values[-1]
                        port_ret = (final_values - initial_value)/initial_value

                        losses = -port_ret

                        var = -np.percentile(losses, 100*(1-alpha))
                        es = losses[losses>=var].mean()

                        self.port_var = var
                        self.port_es = es
                        self.port_losses = losses
                        self.port_values = port_values

                        return var, es, port_ret

NameError: name 'alpha' is not defined

In [ ]:
class risk_analysis:
    def __init__(self,data,tau_annual,alpha):
        self.data = data
        self.tau_annual = tau_annual
        self.alpha = alpha
        '''
        Risk Analysis
        :param data: dataset
        :param tau_annual: 预期年化收益率
        :return:
            没想好怎么做，先放着吧
        '''
        def semi_variance(data,ret, tau):
            downside_returns = data['ret'<tau]
            if len(downside_returns)==0:
                return 0
            return np.mean((downside_returns-tau)**2)
        def shortfall_probability(data,ret, tau):
            return np.mean(data['ret']<tau)
        def VaR(data,alpha,method):
            if method=='historical_simulation':
                return -np.percentile(data['ret'],100*(1-alpha))
            elif method=='parametric_method':
                mu = np.mean(data['ret'])
                sigma = np.std(data['ret'])
                z = stats.norm.ppf(1-alpha)
                var = -(mu-z*sigma)
                return var
            elif method == 'MonteCarlo':
                montecarlo = MonteCarlo()
                return montecarlo.calculate_port_var(alpha)
            else:
                print("Unsupported Method")

        def ES(self, alpha):
            var = self.var_historical(1-alpha)
            losses = self.returns[self.returns<=-var]
            es = -np.mean(losses) if len(losses)>0 else 0
            return es

        def report(self):
        print("="*60)
        print("downside risk index analysis report")
        print("="*60)
        print(f'数据点数')
        print(f"目标收益率：{self.tau_annual:.4f}")
        print(f"平均收益率:{np.mean(self.returns):.4f}")
        print(f"标准差:{np.std(self.returns):.4f}")

        print("="*60)
        print("downside risk index")
        print("="*60)

        print(f"Semi Variance:{self.semi_variance():.4f}")
        print(f"shortfall probability:{self.shortfall_probability():.4f}")

        print("="*60)
        print("\n value in risk")
        print("="*60)

        for alpha in [0.01,0.05]:
            var_hist = self.var_historical(1-alpha)
            var_param = self.var_parametric(1-alpha)
            es = self.expected_shortfall(1-alpha)

            print(f"\n confidence level:{1-alpha:.2f}%")
            print(f" historical VaR:{var_hist:.4f}")
            print(f" parametric VaR:{var_param:.4f}")
            print(f" expected shortfall:{es:.4f}")
            print(f"ES/VaR:{es/var_hist:.4f}")
            print("="*60)


In [26]:
class Analysis:
    def __init__(self,path):
        self.path = path
    def _get_dataset(self):
        if not os.path.exists(self.path):
            print("Wrong Path")
            return None
        else:
            if os.endswith(self.path,".csv"):
                data = pd.read_csv(self.path)
            elif os.endswith(self.path,".xlsx"):
                data = pd.read_excel(self.path)
            elif os.endswith(self.path,".parquet"):
                data = pd.read_parquet(self.path)
            elif os.endswith(self.path,".pkl"):
                data = pd.read_pickle(self.path)
            else:
                print('Unsupported File Format')
                return None
    def _data_cleaning(self,data):
        for col in data.columns:
            data[col] = pd.to_numeric(data[col], errors='coerce')

        data_mask = data.isnull().any(axis=1)
        data_cleaned = data[~data_mask]
        return data_cleaned

    def _calculate_returns(self,close,data):
        '''

        :param close: 收盘价
        :param data:
        :return:
        '''
        if data['ret'].empty:
            data['ret'] =data['close'].pct_change()



SyntaxError: incomplete input (4050344691.py, line 66)